# Introduction

A MultiIndex is an index with multiple, hierarchical levels. We use a multiindex for the column headers of a 
dataframe. This allows each column to have multiple keys associated with it--one key for each level. 

This makes it easy to associate multiple identifiers with a single column.  To show what this looks like we'll look at phosphoproteomics data in the endometrial cancer dataset.

In [1]:
# Throughout the tutorial we will be using the endometrial and colon dataset
import cptac
cptac.download(dataset="endometrial", version="latest")
cptac.download(dataset="colon", version="latest")

True

In [2]:
en = cptac.Endometrial()
phospho = en.get_phosphoproteomics()
phospho.head()

Name        AAAS                  AACS  AAED1 AAGAB          AAK1         \
Site        S495    S541   Y485   S618    S12  S310   S311    S14    S18   
Sample_ID                                                                  
S001         NaN     NaN    NaN -0.881 -1.810   NaN    NaN    NaN -0.242   
S002         NaN     NaN    NaN    NaN  0.084   NaN    NaN -1.110 -0.383   
S003      -0.202     NaN    NaN    NaN -1.880   NaN    NaN    NaN  0.382   
S005      -0.002     NaN -0.407    NaN    NaN   NaN    NaN    NaN    NaN   
S006       0.556 -0.0461    NaN    NaN  0.941   NaN  0.429  0.362  0.697   

Name               ...     ZZZ3                                            \
Site          S20  ...     S397 S411 S420   S424     S426 S468   S89 T415   
Sample_ID          ...                                                      
S001      -0.2420  ...  0.18400  NaN  NaN    NaN -0.20500  NaN   NaN  NaN   
S002      -1.0900  ... -0.17100  NaN  NaN -0.393 -0.17100  NaN  0.29  NaN   
S003      -0.0416  ...      NaN  NaN  NaN    NaN      NaN  NaN   NaN  NaN   
S005      -0.5550  ...  0.13970  NaN  NaN    NaN -0.55900  NaN   NaN  NaN   
S006      -0.0529  ... -0.15875  NaN  NaN  0.196  0.06175  NaN   NaN  NaN   

Name                       
Site         T418    Y399  
Sample_ID                  
S001          NaN     NaN  
S002       0.1605 -0.0635  
S003          NaN     NaN  
S005          NaN  0.2980  
S006          NaN -0.2900  

[5 rows x 73212 columns]

The first level of keys is the name of the gene associated with the column of data. The second level is the site of acetylation. Each column has its own Name and Site associated with it. 
Normally, as above, keys that repeat for adjacent columns are not displayed, to make it easier to read the dataframe. For example, in the table above the first 7 columns are all for the A2M gene, but the label is only shown for the first column. 

# Join functions with multiindices 
The join functions have been written to handle multiindices. More information on the join functions can be found in the joining_dataframes tutorial. 
An example of joining a multiindexed dataframe (in this case phosphoproteomics) with a non multiindexed dataframe (in this case CNV) is below. 

In [3]:
phospho_and_CNV = en.join_omics_to_omics(df1_name="CNV", df2_name="phosphoproteomics")
phospho_and_CNV.head()

Name      A1BG_CNV A1BG-AS1_CNV A1CF_CNV A2M_CNV A2M-AS1_CNV A2ML1_CNV  \
Site           NaN          NaN      NaN     NaN         NaN       NaN   
Sample_ID                                                                
S001          0.00         0.00    -0.01    0.00        0.00      0.00   
S002          0.01         0.01     0.00    0.00        0.00      0.00   
S003          0.01         0.01     0.00    0.00        0.00      0.00   
S005         -0.04        -0.04     0.36   -0.05       -0.05     -0.05   
S006          0.85         0.85     0.16   -0.41       -0.41     -0.41   

Name      A2MP1_CNV A3GALT2_CNV A4GALT_CNV A4GNT_CNV  ...  \
Site            NaN         NaN        NaN       NaN  ...   
Sample_ID                                             ...   
S001           0.00        0.00       0.01     -0.01  ...   
S002           0.00        0.00       0.01      0.00  ...   
S003           0.00        0.01       0.00      0.00  ...   
S005          -0.05       -0.04      -0.04     -0.05  ...   
S006          -0.41       -0.12       0.15      0.19  ...   

Name      ZZZ3_phosphoproteomics                                            \
Site                        S397 S411 S420   S424     S426 S468   S89 T415   
Sample_ID                                                                    
S001                     0.18400  NaN  NaN    NaN -0.20500  NaN   NaN  NaN   
S002                    -0.17100  NaN  NaN -0.393 -0.17100  NaN  0.29  NaN   
S003                         NaN  NaN  NaN    NaN      NaN  NaN   NaN  NaN   
S005                     0.13970  NaN  NaN    NaN -0.55900  NaN   NaN  NaN   
S006                    -0.15875  NaN  NaN  0.196  0.06175  NaN   NaN  NaN   

Name                       
Site         T418    Y399  
Sample_ID                  
S001          NaN     NaN  
S002       0.1605 -0.0635  
S003          NaN     NaN  
S005          NaN  0.2980  
S006          NaN -0.2900  

[5 rows x 101269 columns]

Since the CNV dataframe doesn't have the level "Site", it is filled in with NANs, so that it can be joined to the phosphoproteomics dataframe. 

# How to select from multiindex


## Selecting based on all levels
We can select single columns by passing the proper keys for all levels of the multiindex. For example, to get the acetylproteomics for A2M at site K1176, we'd do the following:

In [4]:
acetyl = en.get_acetylproteomics()
all_levels_selection = acetyl["A2M"]["K1176"]
all_levels_selection.head(10)

Sample_ID
S001    1.080
S002    0.477
S003      NaN
S005   -0.608
S006    1.630
S007   -0.210
S008   -0.175
S009   -0.223
S010   -1.120
S011      NaN
Name: K1176, dtype: float64

## Selecting based on one level
We can easily select multiple columns from our multiindex dataframe, based on just the "Name" level of the multiindex:

In [5]:
gene1_filter = acetyl.columns.get_level_values("Name").str.startswith("AA") # Select all columns where the gene starts with "AA". This will grab every column where the key "Name" starts with AA
gene1_data = acetyl.loc[:, gene1_filter]
gene1_data.head()

Name      AACS  AAGAB     AAK1 AARS AASDH AASS               AATF    
Site      K391   K290     K201 K338  K765 K138 K539    K93   K380 K74
Sample_ID                                                            
S001       NaN  0.461      NaN  NaN   NaN  NaN  NaN    NaN  1.300 NaN
S002       NaN  1.770      NaN  NaN   NaN  NaN  NaN    NaN  0.652 NaN
S003       NaN -0.815 -0.00573  NaN   NaN  NaN  NaN -0.305  0.398 NaN
S005       NaN    NaN      NaN  NaN   NaN  NaN  NaN    NaN    NaN NaN
S006       NaN  0.205      NaN  NaN   NaN  NaN  NaN    NaN  0.244 NaN

## Selecting based on a different level of the multiindex
We can also select based on one of the inner levels of the multiindex. For example, to get all sites from all genes where tyrosine is phosphorylated:

In [6]:
y_site_filter = phospho.columns.get_level_values("Site").str.contains("Y") # Create a boolean filter selecting all columns where the Site level contains a "Y"

y_sites = phospho.loc[:, y_site_filter] # Select the columns
y_sites.head()

Name        AAAS    AAK1   ABCC1   ABI1 ABI2 ABI3   ABL1   ABL2 ABLIM1         \
Site        Y485    Y687    Y920   Y213 Y213 Y361   Y204    Y80   Y357   Y439   
Sample_ID                                                                       
S001         NaN     NaN  0.4340 -0.591  NaN  NaN -0.759    NaN    NaN  0.312   
S002         NaN     NaN -0.5110    NaN  NaN  NaN -1.210  0.321    NaN    NaN   
S003         NaN     NaN     NaN    NaN  NaN  NaN    NaN    NaN    NaN    NaN   
S005      -0.407     NaN  1.2680    NaN  NaN  NaN -0.173    NaN    NaN    NaN   
S006         NaN -0.4315 -0.0685    NaN  NaN  NaN -0.965  0.657    NaN    NaN   

Name       ... ZNF799  ZNF839 ZNF860  ZNRF2 ZRANB2         ZRSR2    ZYX ZZEF1  \
Site       ...   Y430    Y283    Y49   Y111   Y114   Y124   Y347   Y172 Y1795   
Sample_ID  ...                                                                  
S001       ...    NaN     NaN    NaN    NaN    NaN  0.181  0.788 -0.378   NaN   
S002       ...    NaN  0.0684    NaN    NaN    NaN -0.929    NaN -1.230   NaN   
S003       ...    NaN     NaN    NaN    NaN    NaN -0.283  0.675  0.700   NaN   
S005       ...    NaN     NaN    NaN    NaN    NaN -0.123 -0.026 -1.020   NaN   
S006       ...    NaN     NaN    NaN -0.058    NaN  0.109    NaN -0.636   NaN   

Name         ZZZ3  
Site         Y399  
Sample_ID          
S001          NaN  
S002      -0.0635  
S003          NaN  
S005       0.2980  
S006      -0.2900  

[5 rows x 2397 columns]

# How to use DataSet.reduce_multiindex()
To make it easier to work with multi-level indices, we have provide the DataSet.reduce_multiindex function. It can both drop levels from a multiindex, and "flatten" a multi-level index into a single-level index by concatenating the keys from multiple levels into a single key for each column.

## Dropping Levels
We can drop levels based on index or name. We can also drop single or multiple levels at once. 
Note that it will warn you if duplicate column key combinations arise due to dropping levels. 

### Dropping by index or name

In [7]:
en.reduce_multiindex(df=phospho, levels_to_drop="Site").head()

Name,AAAS,AAAS,AAAS,AACS,AAED1,AAGAB,AAGAB,AAK1,AAK1,AAK1,...,ZZZ3,ZZZ3,ZZZ3,ZZZ3,ZZZ3,ZZZ3,ZZZ3,ZZZ3,ZZZ3,ZZZ3
Sample_ID,,,,,,,,,,,,,,,,,,,,,
S001,NaN,NaN,NaN,-0.881,-1.810,NaN,NaN,NaN,-0.242,-0.2420,...,0.18400,NaN,NaN,NaN,-0.20500,NaN,NaN,NaN,NaN,NaN
S002,NaN,NaN,NaN,NaN,0.084,NaN,NaN,-1.110,-0.383,-1.0900,...,-0.17100,NaN,NaN,-0.393,-0.17100,NaN,0.29,NaN,0.1605,-0.0635
S003,-0.202,NaN,NaN,NaN,-1.880,NaN,NaN,NaN,0.382,-0.0416,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
S005,-0.002,NaN,-0.407,NaN,NaN,NaN,NaN,NaN,NaN,-0.5550,...,0.13970,NaN,NaN,NaN,-0.55900,NaN,NaN,NaN,NaN,0.2980
S006,0.556,-0.0461,NaN,NaN,0.941,NaN,0.429,0.362,0.697,-0.0529,...,-0.15875,NaN,NaN,0.196,0.06175,NaN,NaN,NaN,NaN,-0.2900


### Dropping single or multiple at once
By passing a list (or array-like) to levels_to drop, we can drop multiple levels of the multiindex at the same time. Note that we must leave at least one existing level. 

We will show this with the colon data.

In [8]:
colon = cptac.Colon()
phospho = colon.get_phosphoproteomics()
phospho.head()

Name           AAAS                AAED1  AAGAB          AAK1                \
Site           S495   S525   S541    S12   S310   S311    S20    S21    S26   
Database_ID  Q9NRG9 Q9NRG9 Q9NRG9 Q7RTV5 Q6PD74 Q6PD74 Q2M2I8 Q2M2I8 Q2M2I8   
Sample_ID                                                                     
S002            NaN    NaN -0.240 -0.460    NaN    NaN -0.231    NaN    NaN   
S003        -0.3650    NaN    NaN -0.424 -0.015 -0.015    NaN -0.485    NaN   
S004         0.7250  0.137  0.137    NaN    NaN    NaN    NaN    NaN    NaN   
S005         0.2265    NaN    NaN -1.278  0.403  0.075 -0.223 -0.701    NaN   
S006         0.5600    NaN    NaN -0.382    NaN    NaN    NaN -0.259    NaN   

Name                ...  ZZEF1                                ZZZ3         \
Site          S618  ...  S1501   S1518  S1537  S1540  T1521   S113   S391   
Database_ID Q2M2I8  ... O43149  O43149 O43149 O43149 O43149 Q8IYH5 Q8IYH5   
Sample_ID           ...                                                     
S002           NaN  ...    NaN -0.6750 -1.404 -1.404    NaN -0.572    NaN   
S003           NaN  ...    NaN -0.2875  0.222  0.222 -0.701  0.624    NaN   
S004           NaN  ... -0.147 -0.1470    NaN    NaN    NaN    NaN    NaN   
S005           NaN  ... -0.041  0.0430  0.554  0.554  0.127  1.263    NaN   
S006           NaN  ...    NaN -0.1290 -0.919 -0.919    NaN  0.032    NaN   

Name                              
Site          S606    S90    S91  
Database_ID Q8IYH5 Q8IYH5 Q8IYH5  
Sample_ID                         
S002         0.205    NaN    NaN  
S003           NaN    NaN    NaN  
S004           NaN    NaN  -0.03  
S005           NaN    NaN    NaN  
S006         0.026    NaN    NaN  

[5 rows x 31339 columns]

In [9]:
#Drop evey level except Database_ID
drop = ["Name", "Site"]
colon.reduce_multiindex(df=phospho, levels_to_drop=drop)

Database_ID,Q9NRG9,Q9NRG9,Q9NRG9,Q7RTV5,Q6PD74,Q6PD74,Q2M2I8,Q2M2I8,Q2M2I8,Q2M2I8,...,O43149,O43149,O43149,O43149,O43149,Q8IYH5,Q8IYH5,Q8IYH5,Q8IYH5,Q8IYH5
Sample_ID,,,,,,,,,,,,,,,,,,,,,
S002,NaN,NaN,-0.2400,-0.460,NaN,NaN,-0.231,NaN,NaN,NaN,...,NaN,-0.6750,-1.404,-1.404,NaN,-0.572,NaN,0.205,NaN,NaN
S003,-0.3650,NaN,NaN,-0.424,-0.015,-0.015,NaN,-0.485,NaN,NaN,...,NaN,-0.2875,0.222,0.222,-0.701,0.624,NaN,NaN,NaN,NaN
S004,0.7250,0.137,0.1370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.147,-0.1470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03
S005,0.2265,NaN,NaN,-1.278,0.403,0.075,-0.223,-0.701,NaN,NaN,...,-0.041,0.0430,0.554,0.554,0.127,1.263,NaN,NaN,NaN,NaN
S006,0.5600,NaN,NaN,-0.382,NaN,NaN,NaN,-0.259,NaN,NaN,...,NaN,-0.1290,-0.919,-0.919,NaN,0.032,NaN,0.026,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S206,-0.3240,-0.482,-0.1945,0.158,NaN,NaN,0.102,-0.461,NaN,NaN,...,NaN,0.3630,1.549,1.549,NaN,NaN,NaN,NaN,0.379,NaN
S207,-1.0220,NaN,NaN,NaN,NaN,NaN,0.670,0.342,NaN,NaN,...,NaN,0.5570,-0.004,-0.004,0.546,NaN,NaN,-0.090,0.043,NaN
S208,-0.6120,NaN,NaN,-0.404,NaN,-0.493,NaN,-0.072,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.179,NaN,NaN,NaN


## Combining levels (Flattening)

We can combine levels of a multiindexed dataframe. When combined the levels will be sepereated by an underscore, by default. Note that we could specify a different seperator.

In [10]:
colon.reduce_multiindex(df=phospho, flatten=True)

Name,AAAS_S495_Q9NRG9,AAAS_S525_Q9NRG9,AAAS_S541_Q9NRG9,AAED1_S12_Q7RTV5,AAGAB_S310_Q6PD74,AAGAB_S311_Q6PD74,AAK1_S20_Q2M2I8,AAK1_S21_Q2M2I8,AAK1_S26_Q2M2I8,AAK1_S618_Q2M2I8,...,ZZEF1_S1501_O43149,ZZEF1_S1518_O43149,ZZEF1_S1537_O43149,ZZEF1_S1540_O43149,ZZEF1_T1521_O43149,ZZZ3_S113_Q8IYH5,ZZZ3_S391_Q8IYH5,ZZZ3_S606_Q8IYH5,ZZZ3_S90_Q8IYH5,ZZZ3_S91_Q8IYH5
Sample_ID,,,,,,,,,,,,,,,,,,,,,
S002,NaN,NaN,-0.2400,-0.460,NaN,NaN,-0.231,NaN,NaN,NaN,...,NaN,-0.6750,-1.404,-1.404,NaN,-0.572,NaN,0.205,NaN,NaN
S003,-0.3650,NaN,NaN,-0.424,-0.015,-0.015,NaN,-0.485,NaN,NaN,...,NaN,-0.2875,0.222,0.222,-0.701,0.624,NaN,NaN,NaN,NaN
S004,0.7250,0.137,0.1370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.147,-0.1470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03
S005,0.2265,NaN,NaN,-1.278,0.403,0.075,-0.223,-0.701,NaN,NaN,...,-0.041,0.0430,0.554,0.554,0.127,1.263,NaN,NaN,NaN,NaN
S006,0.5600,NaN,NaN,-0.382,NaN,NaN,NaN,-0.259,NaN,NaN,...,NaN,-0.1290,-0.919,-0.919,NaN,0.032,NaN,0.026,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S206,-0.3240,-0.482,-0.1945,0.158,NaN,NaN,0.102,-0.461,NaN,NaN,...,NaN,0.3630,1.549,1.549,NaN,NaN,NaN,NaN,0.379,NaN
S207,-1.0220,NaN,NaN,NaN,NaN,NaN,0.670,0.342,NaN,NaN,...,NaN,0.5570,-0.004,-0.004,0.546,NaN,NaN,-0.090,0.043,NaN
S208,-0.6120,NaN,NaN,-0.404,NaN,-0.493,NaN,-0.072,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.179,NaN,NaN,NaN


In [11]:
colon.get_transcriptomics().head()

Name,A1CF,A2M,A4GALT,AAAS,AACS,AADACP1,AADAT,AAED1,AAGAB,AAK1,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
Sample_ID,,,,,,,,,,,,,,,,,,,,,
S001,10.1984,13.8117,7.0224,9.4115,9.7432,4.5850,8.4635,7.6221,11.2336,8.3219,...,10.1046,10.5765,10.5294,7.0553,8.4263,8.6865,10.0307,9.1699,10.2784,10.4263
S002,10.1586,12.6434,6.7279,8.9972,9.9084,4.7004,8.2046,7.0768,10.5362,9.6402,...,9.6883,9.3264,9.2808,7.1799,9.2946,9.1799,9.8408,10.5622,10.4512,10.4305
S003,9.2119,15.0040,6.8580,9.7056,9.1997,5.0000,8.4676,7.1293,10.9337,9.5661,...,9.8424,9.8611,10.2107,7.6582,8.7415,9.0741,10.3309,10.0512,10.2703,10.5430
S004,10.7322,13.9284,4.9542,9.1497,8.9425,6.0224,7.4179,7.7074,10.7194,9.3376,...,9.6812,9.0795,9.5118,8.9159,9.9542,10.0168,10.3365,9.9307,10.2538,10.8925
S005,9.7160,12.9963,6.3576,9.6202,9.2738,4.1699,7.0980,5.9069,10.2252,9.7944,...,10.1344,9.3174,10.0238,7.4429,8.7381,9.6165,10.3848,11.3066,11.2174,10.7616


### When flatteing levels , NaNs will automitically be dropped

In [12]:
phospho_and_CNV = en.join_omics_to_omics(df1_name="CNV", df2_name="phosphoproteomics")
phospho_and_CNV.head()

Name      A1BG_CNV A1BG-AS1_CNV A1CF_CNV A2M_CNV A2M-AS1_CNV A2ML1_CNV  \
Site           NaN          NaN      NaN     NaN         NaN       NaN   
Sample_ID                                                                
S001          0.00         0.00    -0.01    0.00        0.00      0.00   
S002          0.01         0.01     0.00    0.00        0.00      0.00   
S003          0.01         0.01     0.00    0.00        0.00      0.00   
S005         -0.04        -0.04     0.36   -0.05       -0.05     -0.05   
S006          0.85         0.85     0.16   -0.41       -0.41     -0.41   

Name      A2MP1_CNV A3GALT2_CNV A4GALT_CNV A4GNT_CNV  ...  \
Site            NaN         NaN        NaN       NaN  ...   
Sample_ID                                             ...   
S001           0.00        0.00       0.01     -0.01  ...   
S002           0.00        0.00       0.01      0.00  ...   
S003           0.00        0.01       0.00      0.00  ...   
S005          -0.05       -0.04      -0.04     -0.05  ...   
S006          -0.41       -0.12       0.15      0.19  ...   

Name      ZZZ3_phosphoproteomics                                            \
Site                        S397 S411 S420   S424     S426 S468   S89 T415   
Sample_ID                                                                    
S001                     0.18400  NaN  NaN    NaN -0.20500  NaN   NaN  NaN   
S002                    -0.17100  NaN  NaN -0.393 -0.17100  NaN  0.29  NaN   
S003                         NaN  NaN  NaN    NaN      NaN  NaN   NaN  NaN   
S005                     0.13970  NaN  NaN    NaN -0.55900  NaN   NaN  NaN   
S006                    -0.15875  NaN  NaN  0.196  0.06175  NaN   NaN  NaN   

Name                       
Site         T418    Y399  
Sample_ID                  
S001          NaN     NaN  
S002       0.1605 -0.0635  
S003          NaN     NaN  
S005          NaN  0.2980  
S006          NaN -0.2900  

[5 rows x 101269 columns]

In [13]:
#Notice that the NaNs have been dropped
en.reduce_multiindex(df=phospho_and_CNV, flatten=True).head()

Name,A1BG_CNV,A1BG-AS1_CNV,A1CF_CNV,A2M_CNV,A2M-AS1_CNV,A2ML1_CNV,A2MP1_CNV,A3GALT2_CNV,A4GALT_CNV,A4GNT_CNV,...,ZZZ3_phosphoproteomics_S397,ZZZ3_phosphoproteomics_S411,ZZZ3_phosphoproteomics_S420,ZZZ3_phosphoproteomics_S424,ZZZ3_phosphoproteomics_S426,ZZZ3_phosphoproteomics_S468,ZZZ3_phosphoproteomics_S89,ZZZ3_phosphoproteomics_T415,ZZZ3_phosphoproteomics_T418,ZZZ3_phosphoproteomics_Y399
Sample_ID,,,,,,,,,,,,,,,,,,,,,
S001,0.00,0.00,-0.01,0.00,0.00,0.00,0.00,0.00,0.01,-0.01,...,0.18400,NaN,NaN,NaN,-0.20500,NaN,NaN,NaN,NaN,NaN
S002,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,...,-0.17100,NaN,NaN,-0.393,-0.17100,NaN,0.29,NaN,0.1605,-0.0635
S003,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
S005,-0.04,-0.04,0.36,-0.05,-0.05,-0.05,-0.05,-0.04,-0.04,-0.05,...,0.13970,NaN,NaN,NaN,-0.55900,NaN,NaN,NaN,NaN,0.2980
S006,0.85,0.85,0.16,-0.41,-0.41,-0.41,-0.41,-0.12,0.15,0.19,...,-0.15875,NaN,NaN,0.196,0.06175,NaN,NaN,NaN,NaN,-0.2900
